*The dataset used in this example is distributed at https://doi.org/10.11588/data/TJNQZG under the [CC-BY-SA-4.0](https://creativecommons.org/licenses/by-sa/4.0/) license.*

# Steps in a minimal workflow with AFwizard


This minimal workflow explains the most important steps with links to more detailed description in the library documentation. The sample data set used offers the original data and important data that were created in the course of this minimal workflow, such that users may always refer to these solutions. 

All data are stored and new data created in the top level directory, therefore, explanations in the following codeblocks do not use subdirectories and paths. However, using subdirectories for different kinds of data is useful in the course of intensive work with the AFwizard. A sub directory for the final output will be created automatically if no option is given (see below "Step 5: Adaptive Filtering") 

First of all, any Jupyter notebook that uses AFwizard needs to import the `afwizard` library. Using the alias `af` in the code below allows us to use a short form in the following steps instead of `afwizard`.

In [ ]:
import afwizard as af

## Step 1: Prepare original point cloud and segment file

_Detailed explanations can be found in the documentation on [Working with LIDAR datasets in AFwizard](https://afwizard.readthedocs.io/en/latest/datasets.html)._

Load original point cloud (sample data: `nkd_pcl_epsg6670.laz`) of the whole region: 

In [ ]:
ds = af.DataSet(filename="nkd_pcl_epsg6670.laz", spatial_reference="EPSG:6670")

Remove existing classification from the point cloud (optional step).

In [ ]:
ds = af.remove_classification(ds)

Load segmentation layer and its spatial reference (sample data: `nkd_sgm.geojson`, EPSG:6670) where all segments cover the whole region and are classified according to environmental characteristics:

In [ ]:
segmentation_overlay = af.load_segmentation(
    "nkd_sgm.geojson", spatial_reference="EPSG:6670"
)

## Step 2: Create spatial samples (point clouds) for segment classes

_Detailed explanations can be found in the documentation on [Working with LIDAR datasets in AFwizard](https://afwizard.readthedocs.io/en/latest/datasets.html)._

We will now prepare samples from the point cloud such that each spatial sample represents the typical features of one segment class. The sample data set contains segment classes "paddy", "ridge", "slope", "valley". 

The step of creating a spatial sample is called `restrict`, it is performed in a Graphical User Interface that is called as follows: 

In [ ]:
rds = ds.restrict(segmentation_overlay=segmentation_overlay)

If the optional step of removing classification of point clouds has not been performed, the lower left part will inform about how many points of which class the data set contains. 

Clicking on the button "Visualize" on the left side gives the option to load a standard visualisation of the area which can be modified with some options below. The zoom level may be changed on the right side. Clicking on the polygon in the left half of the map enables the user to create a subselection of a segment as representative of a segment class. We create a spatial sample for the class "valley". After clicking into the segment and finishing the polygon, we click on finalize and close the GUI.

The sample set has to be saved; in order not to overwrite the existing sample data `sample_pcl_valley.las`, we call it `sample_pcl_valley_new.las`. 

In [ ]:
saved = rds.save("sample_pcl_valley_new.las", overwrite=False)

We have now created and saved one point cloud data set that is representative for valleys in our original point cloud. The steps under "Step 2" have to be repeated for all other segment classes, such that we save four new files `..._new.las`.

The sample data set already contains one example for each class (`sample_pcl_paddy.las`, `sample_pcl_ridge.las`, `sample_pcl_slope.las`, `sample_pcl_valley.las`), such that the  steps below can be tested with these data sets instead of newly created data sets.

## Step 3: Creating filter pipelines

_Detailed explanations can be found in the documentation on [Working with filter libraries](https://afwizard.readthedocs.io/en/latest/libraries.html)._

We can now use these sample data sets to create filter pipelines from scratch or to modify existing filter pipelines like those shipped with the sample data (`nkd_fpl_paddy_LT.json`, `nkd_fpl_slope_LT.json`, `nkd_fpl_valley_TF.json`). You can alternatively use filter pipelines that are already stored in the filter pipeline repo where filter pipelines can be published by the community (see [Sharing filter pipelines with others](https://afwizard.readthedocs.io/en/latest/libraries.html#Sharing-filter-pipelines-with-others)).

The sample data set ships with three filter pipelines that were created on the basis of the four `.las` files that were created as described in step 2. Since the segments of class "ridge" use identical settings to the segments of class "slope", the same filter pipeline can be used for slopes and ridges, therefore, only the three filter pipelines mentioned above are shipped for four segment classes. 

However, these filter pipelines need an installation of OPALS and LAStools. Therefore, you can only use the filterpipelines if you have access to these tools. In that case, the path to each of these tools has to be set with the respective AFwizard commands.

**To perform these commands as shown in the following two lines, delete the `# ` sign at the beginning of each line and set the correct path to the installations.**

In [ ]:
# af.set_opals_directory([path_to_opals])
# af.set_lastools_directory([path_to_lastools])

In this workflow, we will show the creation and modification of a filter pipeline for segments of the class "slope"; **the steps have to be repeated for all segment classes**.

Regardless of whether we create a new filter pipeline from scratch or modify an existing one, we test the parameter settings with the respective spatial sample. In this workflow we will use the pointcloud sample for "slope" that was shipped with the sample data: `sample_pcl_slope.las` and declare the spatial reference, which is EPSG:6670 in our case.

In [ ]:
dataset = af.DataSet(filename="sample_pcl_slope.las", spatial_reference="EPSG:6670")

### Creating a filter pipeline from scratch

We will now create a filter pipeline for segments of class "slope" from scratch and therefore load the GUI for pipeline tuning. A detailed description of the process in this specific example is given in the documentation of the sample data, here we will only give parameters for two steps.

1. The data set is shown in tab \#0 just as loaded.
2. Raster resolution on the right is set to 0.2, and the hillshade parameters are modified such that the Angle Altitude is 45 degrees (45.30). Clicking the preview button shows tab \#1.
3. Another pipeline stage is selected on the left side; here we choose `lasground_new (LASTools)` with their default values and click the preview button; the result is shown in tab \#2.
4. To add more options to choose a better example, we create a range of parameters for the LASTools in order to later compare the results and select the best option.
    1. The parameter spike will be calculated with 0.4 increments between 0.1 and 0.9.
    2. For each of the thre values, thre parameter steps at 0.6, 1.2, 1.8 will be calculated.
    3. Clicking on the preview button calculated 9 candidates which are shown in tabs \#3 to \#11.
    4. The model in tab \#6 (spike 0.1; step 1.2 -- tab numbers may differ!) seems the best candidate. 
5. The new candidate will be further refined by adding a range to the parameter step from value 0.9 to 1.5 with 0.1 increments; this adds 6 more models which are shown in the tabs \#12 to \#17 after clicking the preview button.

Tabs that are of no use can be deleted with the button "Delete this filtering" on the right side; models that shall be preserved can be saved as point clouds, geotiffs or images using the buttons below the view in the center column. 

A log of all steps is shown below the GUI.

Finally, a setting for the filter pipeline has to be chosen, we decide for tab\#13 (offset 0; spike 0.1; step 1). Chosing the respective tab with these parameters, we can enter meta data in the lower left column and click the button `Finalize`. 

In [ ]:
pipeline = af.pipeline_tuning(dataset)

The filter pipeline name may differ from the file name of the pipeline. -- In our sample data set we have chosen to give the same name to the filter pipeline file name as to the filter pipeline itself (also called `pipeline title`) which is not necessary.

In [ ]:
af.save_filter(pipeline, "nkd_fpl_slope_LT_new.json")

### Modify existing filter pipelines

The `dataset`is loaded as shown at the start of step 3 above. Now we load an existing filter pipeline as `old_variable`; here we will load the filter pipeline for segments of class "slope", which is `nkd_fpl_slope_LT.json` in the sample data set.

In [ ]:
old_pipeline = af.load_filter("nkd_fpl_slope_LT.json")

This pipeline will be edited with the next command just like when a new filter pipeline is adjusted:

In [ ]:
edited_pipeline = af.pipeline_tuning(dataset, pipeline=old_pipeline)

The new version is saved with the following command; we will add "modified" to the original file name in this case:

In [ ]:
af.save_filter(edited_pipeline, "nkd_fpl_slope_LT_modified.json")

## Step 4: Mapping segmentations to filter pipelines

_Detailed explanations can be found in the documentation on [Mapping segmentations to filter pipelines](https://afwizard.readthedocs.io/en/latest/segmentation.html)._

We load the file containing polygons that represent the extent of segments and their segment class (`nkd_sgm.geojson` in the sample data set). 

In [ ]:
segmentation = af.load_segmentation("nkd_sgm.geojson", spatial_reference="EPSG:6670")

With the following command, we make accessible the filter pipelines stored in the working directory. The sample data set ships with only three filter pipelines since two segment classes ("ridge" and "slope") work well with one and the same filter pipeline (`nkd_fpl_slope_LT.json`). In the left column under "libraries", only the working directory is selected, so the center columns shows the three filter pipelines from the sample data set which are stored in the working directory. 

Clicking on one filter pipeline shows the metadata in the right column. The example shows metadata for the slope filter pipeline selected. To select all three pipelines, we hold the `ctrl` key (Windows) or `cmd` key (Mac) and select each of the three filters. A click on the `Finalize` button on top of the right column loads these filters for application in the next step.

In [ ]:
pipelines = af.select_pipelines_from_library()

Segment classes and filter pipelines can now be matched in a GUI. It should be noted that the field to select the class in the `geojson` file may differ from the name "class" which is chosen in the sample data set; likewise, the name of the filter pipeline may differ in the `geojson` file from the file name itself. In the sample data set, both names are identical. 

Clicking the button "Visualize" will load a standard visualization, which is optional. 

The drop down menu on the right side show the first data field in the GeoJSON file in the beginning; in the sample data set it is the field "id" which holds unique identifiers for each individual segment. Clicking on the drop down menu reduces the list below to only four segment classes.

Each segment will get a filter pipeline name (the name that is stored in the filter pipeline file, not the filename) assigned by choosing from the dropdown menu.

When all filter pipelines are assigned to a segment class, clicking on the "Finalize" button closes the GUI.

In [ ]:
assigned_segmentation = af.assign_pipeline(
    ds, segmentation=segmentation, pipelines=pipelines
)

The information will be saved in another GeoJSON file that contains the same data as the original GeoJSON file and two more data fields: A hash value that represents the filter pipeline settings chosen and an informational field that holds the actual name of the filter pipeline. The assigned segmentation file is saved with the command below: 

In [ ]:
assigned_segmentation.save("nkd_sgm_assigned_TF_new.geojson")

The sample data set ships with a file `nkd_sgm_assigned_TF.geojson` that can be used in the following step.

## Step 5: Adaptive filtering

Finally, the adaptive filtering has to be calculated in order to create a new DTM. This is done on a command line/terminal basis, and it is the heart of the AFwizard. Many parameters may be set in this stage, and a more detailed information can be found in the AFwizard documentation page [afwizard](https://afwizard.readthedocs.io/en/latest/execution.html#afwizard). 

Here we chose the bare minimum of options, and this step can be executed with the data provided in the sample data set.

`afwizard --dataset=nkd_pcl_epsg6670.laz --dataset-crs=EPSG:6670 --segmentation=nkd_sgm_assigned_TF.geojson --segmentation-crs=EPSG:6670`

During this step, an output directory is created as a subdirectory of the current working directory -- if no parameter is given for the output data. This directory holds the following files: 

1. The filter pipeline JSON files for complete documentation. In our case, these are three files. 
2. An output log-file `output.log`.
3. The DTM resulting from adaptive filtering:
    1. The point cloud data, in our case `nkd_pcl_epgs6670_filtered.las`
    2. A multiband GeoTIFF file, in our case `nkd_pcl_epgs6670_filtered.tiff`, where band 1 contains the DTM.

This step wraps up the minimal workflow with the Adaptive Filtering Wizard.